In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from moffragmentor import MOF

from moffragmentor.fragmentor.solventlocator import get_floating_solvent_molecules, get_all_bound_solvent_molecules
from moffragmentor.fragmentor.nodelocator import find_node_clusters, create_node_collection
from moffragmentor.fragmentor.linkerlocator import create_linker_collection

In [22]:
mof = MOF.from_cif('/Users/kevinmaikjablonka/Dropbox (LSMO)/reference_set_project/aiida_optimized_2021_01_08_3203_OLD/opt_cifs/RSM0001.cif')

In [59]:
unbound_solvent = get_floating_solvent_molecules(mof)
    # Find nodes
node_result = find_node_clusters(mof, unbound_solvent.indices)
node_collection = create_node_collection(mof, node_result)
    # Find bound solvent
    # 
bound_solvent = get_all_bound_solvent_molecules(mof, node_result.nodes)
    # Filter the linkers (valid linkers have at least two branch points)

2022-04-13 15:03:51.733 | WARNING  | moffragmentor.fragmentor.nodelocator:find_node_clusters:63 - No branch sites found according to branch site definition.             Using now CN=3 sites between metals as branch sites.                This is not consistent with the conventions used in other parts of the code. ̰


[[6, 5, 4]]


In [60]:
node_result.nodes

[{0, 4, 5, 6, 7, 8, 9, 10, 11, 12}]

In [61]:
node_result.branching_indices

{4, 5, 6}

In [62]:
linker_collection, edge_dict = create_linker_collection(
        mof, node_result, node_collection, unbound_solvent
    )

In [63]:
mof.show_structure()

NGLWidget()

In [64]:
bbs = mof.fragment()

2022-04-13 15:03:55.261 | WARNING  | moffragmentor.fragmentor.nodelocator:find_node_clusters:63 - No branch sites found according to branch site definition.             Using now CN=3 sites between metals as branch sites.                This is not consistent with the conventions used in other parts of the code. ̰


[[6, 5, 4]]


In [68]:
bbs.nodes[0].show_molecule()

NGLWidget()